In [ ]:
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
#from langchain.llms import GooglePalm
#from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
from langchain_community.embeddings.openai import OpenAIEmbeddings

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Let's read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents= file_loader.load()
    return documents


In [ ]:
doc = read_doc('./documents/')

In [ ]:
len(doc)

In [ ]:
# divide the document into chunks

def chunk_data(docs, chunk_size= 800, chunk_ovelap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size)
    doc=text_splitter.split_documents(docs)
    return doc



In [ ]:
documents = chunk_data(docs=doc)
documents

In [ ]:
embeddings = OpenAIEmbeddings(api_key  = os.environ['OPENAI_API_KEY'],)


In [ ]:
embeddings

In [ ]:
vectors = embeddings.embed_query('hello')

In [ ]:
len(vectors)
#import os


In [ ]:
pinecone.init(
    api_key=os.environ['PINECONE_API'],
    environment="gcp-starter"
)
index_name="langchain"

In [ ]:
index =Pinecone.from_documents(doc, embeddings,index_name=index_name)

In [ ]:
## Cosine Similarity Retrieve Result from Vector DB
def retrieve_query(query, k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
#from langchain_google_genai import GoogleGenerativeAI

#from langchain_community.llms import VertexAI
#from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
llm = OpenAI(model_name="text-davinci-003", temperature= 0.5)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
### search answers from VectorDB
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents= doc_search, question= query)
    return response



In [ ]:
ad_query ="What is CAGR of Aerospace and defence sector"
answer = retrieve_answers(ad_query)
print(answer)